## ⚽️ Quiz Week 1 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import seaborn as sns

In [7]:
#Import data
EPL = pd.read_excel('Engsoccer2017-18.xlsx')
EPL

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,EPL,2017-11-08 00:00:00,Arsenal,Leicester,4,3,H
1,EPL,2017-12-08 00:00:00,Brighton,Man City,0,2,A
2,EPL,2017-12-08 00:00:00,Chelsea,Burnley,2,3,A
3,EPL,2017-12-08 00:00:00,Crystal Palace,Huddersfield,0,3,A
4,EPL,2017-12-08 00:00:00,Everton,Stoke,1,0,H
...,...,...,...,...,...,...,...
2031,FL2,2018-05-05 00:00:00,Lincoln,Yeovil,1,1,D
2032,FL2,2018-05-05 00:00:00,Mansfield,Crawley Town,1,1,D
2033,FL2,2018-05-05 00:00:00,Notts County,Luton,0,0,D
2034,FL2,2018-05-05 00:00:00,Swindon,Accrington,3,0,H


In [29]:
EPL["Date"] = pd.to_datetime(EPL["Date"])  # Datum korrekt interpretieren

# Neue Spalte "Jahr" hinzufügen
EPL["Year"] = EPL["Date"].dt.year
EPL

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Year
0,EPL,2017-11-08,Arsenal,Leicester,4,3,H,2017
1,EPL,2017-12-08,Brighton,Man City,0,2,A,2017
2,EPL,2017-12-08,Chelsea,Burnley,2,3,A,2017
3,EPL,2017-12-08,Crystal Palace,Huddersfield,0,3,A,2017
4,EPL,2017-12-08,Everton,Stoke,1,0,H,2017
...,...,...,...,...,...,...,...,...
2031,FL2,2018-05-05,Lincoln,Yeovil,1,1,D,2018
2032,FL2,2018-05-05,Mansfield,Crawley Town,1,1,D,2018
2033,FL2,2018-05-05,Notts County,Luton,0,0,D,2018
2034,FL2,2018-05-05,Swindon,Accrington,3,0,H,2018


In [67]:
#How many EPL games from this season were played in 2018?
Games18 = ((EPL["Year"] > 2017) & (EPL["Div"] == "EPL")).sum()
print(Games18)

171


In [95]:
#Which team scored the highest number of goals while playing at home in the first half of the season?
df = EPL[['Div','HomeTeam','FTHG','Year']]

goals_max = df.groupby(['HomeTeam','Year'])[['FTHG']].sum().sort_values(by='FTHG',ascending=False).reset_index().head(10)
goals_max

,HomeTeam,Year,FTHG
0,Luton,2017,37
1,Man City,2017,36
2,Hull,2017,30
3,Arsenal,2018,29
4,Rotherham,2017,27
5,Fulham,2018,27
6,Wycombe,2017,27
7,Notts County,2017,25
8,Wolves,2017,25
9,Luton,2018,25


In [190]:
#Which team conceded the highest number of goals while playing away in the first half of the season?
df1 = EPL[['Div','AwayTeam','FTHG','Year']]

conceded_max = df1.groupby(['AwayTeam','Year','Div'])[['FTHG']].sum().sort_values(by='FTHG',ascending=False).reset_index().head(10)
conceded_max

,AwayTeam,Year,Div,FTHG
0,Yeovil,2017,FL2,33
1,Southend,2017,FL1,28
2,Stoke,2017,EPL,27
3,Chesterfield,2017,FL2,27
4,Oldham,2017,FL1,26
5,Northampton,2017,FL1,26
6,Nott'm Forest,2017,FLCH,25
7,Bristol Rvs,2017,FL1,25
8,West Ham,2017,EPL,24
9,Rochdale,2017,FL1,24


## Q4 Which of the following teams had the smallest difference between their win percentage and Pythagorean expectation in the first half of the season?

In [108]:
#First, creating a df with only the first half of the season (year < 2018)
half1 = EPL[EPL.Year < 2018]
half1

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Year
0,EPL,2017-11-08,Arsenal,Leicester,4,3,H,2017
1,EPL,2017-12-08,Brighton,Man City,0,2,A,2017
2,EPL,2017-12-08,Chelsea,Burnley,2,3,A,2017
3,EPL,2017-12-08,Crystal Palace,Huddersfield,0,3,A,2017
4,EPL,2017-12-08,Everton,Stoke,1,0,H,2017
...,...,...,...,...,...,...,...,...
1777,FL2,2017-12-30,Grimsby,Accrington,0,3,A,2017
1778,FL2,2017-12-30,Lincoln,Forest Green,2,1,H,2017
1779,FL2,2017-12-30,Port Vale,Luton,4,0,H,2017
1780,FL2,2017-12-30,Swindon,Notts County,1,0,H,2017


In [112]:
#We first identify whether the results was a win for the home team, the away team or a draw. We also create the counting varible.

half1['hwinvalue']=np.where(half1['FTR']=='H',1,np.where(half1['FTR']=='D',0.5,0))
half1['awinvalue'] = np.where(half1['FTR'] == 'A',1,np.where(half1['FTR']=='D',0.5,0))
half1['count']=1  
half1

/var/folders/0q/s7fj151s553_0czdxm4mtj680000gn/T/ipykernel_15273/1897019737.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  half1['hwinvalue']=np.where(half1['FTR']=='H',1,np.where(half1['FTR']=='D',0.5,0))
/var/folders/0q/s7fj151s553_0czdxm4mtj680000gn/T/ipykernel_15273/1897019737.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  half1['awinvalue'] = np.where(half1['FTR'] == 'A',1,np.where(half1['FTR']=='D',0.5,0))
/var/folders/0q/s7fj151s553_0czdxm4mtj680000gn/T/ipykernel_15273/1897019737.py:5: Sett

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Year,hwinvalue,awinvalue,count
0,EPL,2017-11-08,Arsenal,Leicester,4,3,H,2017,1.0,0.0,1
1,EPL,2017-12-08,Brighton,Man City,0,2,A,2017,0.0,1.0,1
2,EPL,2017-12-08,Chelsea,Burnley,2,3,A,2017,0.0,1.0,1
3,EPL,2017-12-08,Crystal Palace,Huddersfield,0,3,A,2017,0.0,1.0,1
4,EPL,2017-12-08,Everton,Stoke,1,0,H,2017,1.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
1777,FL2,2017-12-30,Grimsby,Accrington,0,3,A,2017,0.0,1.0,1
1778,FL2,2017-12-30,Lincoln,Forest Green,2,1,H,2017,1.0,0.0,1
1779,FL2,2017-12-30,Port Vale,Luton,4,0,H,2017,1.0,0.0,1
1780,FL2,2017-12-30,Swindon,Notts County,1,0,H,2017,1.0,0.0,1


In [116]:
#Once again, we have to create separate dfs to calculate home team and away team performance.

Enghome = half1.groupby(['HomeTeam','Div'])[['count','hwinvalue','FTHG','FTAG']].sum().reset_index()
Enghome = Enghome.rename(columns={'HomeTeam':'team','count':'Ph','FTHG':'FTHGh','FTAG':'FTAGh'})
Enghome

,team,Div,Ph,hwinvalue,FTHGh,FTAGh
0,AFC Wimbledon,FL1,12,4.5,11,17
1,Accrington,FL2,11,7.0,19,12
2,Arsenal,EPL,10,8.5,25,10
3,Aston Villa,FLCH,12,8.5,17,10
4,Barnet,FL2,12,4.5,10,14
...,...,...,...,...,...,...
87,West Ham,EPL,9,4.0,10,14
88,Wigan,FL1,13,10.0,21,4
89,Wolves,FLCH,12,10.0,25,10
90,Wycombe,FL2,13,8.5,27,18


In [118]:
#Now we create the away team results df

Engaway = half1.groupby('AwayTeam')[['count','awinvalue','FTHG','FTAG']].sum().reset_index()
Engaway = Engaway.rename(columns={'AwayTeam':'team','count':'Pa','FTHG':'FTHGa','FTAG':'FTAGa'})
Engaway

,team,Pa,awinvalue,FTHGa,FTAGa
0,AFC Wimbledon,12,4.5,14,9
1,Accrington,13,7.0,18,19
2,Arsenal,11,5.0,16,13
3,Aston Villa,13,6.5,14,16
4,Barnet,13,3.0,23,15
...,...,...,...,...,...
87,West Ham,11,3.0,24,12
88,Wigan,11,8.5,8,28
89,Wolves,13,10.0,10,22
90,Wycombe,12,6.0,16,17


In [122]:
#Merging the two dfs

Eng17 = pd.merge(Enghome, Engaway, on=['team'])
Eng17

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa
0,AFC Wimbledon,FL1,12,4.5,11,17,12,4.5,14,9
1,Accrington,FL2,11,7.0,19,12,13,7.0,18,19
2,Arsenal,EPL,10,8.5,25,10,11,5.0,16,13
3,Aston Villa,FLCH,12,8.5,17,10,13,6.5,14,16
4,Barnet,FL2,12,4.5,10,14,13,3.0,23,15
...,...,...,...,...,...,...,...,...,...,...
87,West Ham,EPL,9,4.0,10,14,11,3.0,24,12
88,Wigan,FL1,13,10.0,21,4,11,8.5,8,28
89,Wolves,FLCH,12,10.0,25,10,13,10.0,10,22
90,Wycombe,FL2,13,8.5,27,18,12,6.0,16,17


In [124]:
#Sum the results by home and away measures to get the team overall performance for the season

Eng17['W'] = Eng17['hwinvalue']+Eng17['awinvalue']
Eng17['G'] = Eng17['Ph']+Eng17['Pa']
Eng17['GF'] = Eng17['FTHGh']+Eng17['FTAGa']
Eng17['GA'] = Eng17['FTAGh']+Eng17['FTHGa']
Eng17

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa,W,G,GF,GA
0,AFC Wimbledon,FL1,12,4.5,11,17,12,4.5,14,9,9.0,24,20,31
1,Accrington,FL2,11,7.0,19,12,13,7.0,18,19,14.0,24,38,30
2,Arsenal,EPL,10,8.5,25,10,11,5.0,16,13,13.5,21,38,26
3,Aston Villa,FLCH,12,8.5,17,10,13,6.5,14,16,15.0,25,33,24
4,Barnet,FL2,12,4.5,10,14,13,3.0,23,15,7.5,25,25,37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,West Ham,EPL,9,4.0,10,14,11,3.0,24,12,7.0,20,22,38
88,Wigan,FL1,13,10.0,21,4,11,8.5,8,28,18.5,24,49,12
89,Wolves,FLCH,12,10.0,25,10,13,10.0,10,22,20.0,25,47,20
90,Wycombe,FL2,13,8.5,27,18,12,6.0,16,17,14.5,25,44,34


In [126]:
#Creating wpc and Pythagorean Expectation

Eng17['wpc']=Eng17['W']/Eng17['G']
Eng17['pyth']=Eng17['GF']**2/(Eng17['GF']**2+Eng17['GA']**2)
Eng17

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa,W,G,GF,GA,wpc,pyth
0,AFC Wimbledon,FL1,12,4.5,11,17,12,4.5,14,9,9.0,24,20,31,0.375000,0.293902
1,Accrington,FL2,11,7.0,19,12,13,7.0,18,19,14.0,24,38,30,0.583333,0.616041
2,Arsenal,EPL,10,8.5,25,10,11,5.0,16,13,13.5,21,38,26,0.642857,0.681132
3,Aston Villa,FLCH,12,8.5,17,10,13,6.5,14,16,15.0,25,33,24,0.600000,0.654054
4,Barnet,FL2,12,4.5,10,14,13,3.0,23,15,7.5,25,25,37,0.300000,0.313440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,West Ham,EPL,9,4.0,10,14,11,3.0,24,12,7.0,20,22,38,0.350000,0.251037
88,Wigan,FL1,13,10.0,21,4,11,8.5,8,28,18.5,24,49,12,0.770833,0.943418
89,Wolves,FLCH,12,10.0,25,10,13,10.0,10,22,20.0,25,47,20,0.800000,0.846685
90,Wycombe,FL2,13,8.5,27,18,12,6.0,16,17,14.5,25,44,34,0.580000,0.626132


In [137]:
#New column for difference between wpc and pyth
Eng17['difference'] = Eng17['wpc']-Eng17['pyth']
Eng17.sort_values(by='difference')

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa,W,G,GF,GA,wpc,pyth,difference
88,Wigan,FL1,13,10.0,21,4,11,8.5,8,28,18.5,24,49,12,0.770833,0.943418,-0.172585
23,Chelsea,EPL,11,8.5,21,7,10,7.0,7,18,15.5,21,39,14,0.738095,0.885847,-0.147752
47,Luton,FL2,12,9.5,37,11,13,8.5,12,21,18.0,25,58,23,0.720000,0.864115,-0.144115
83,Tottenham,EPL,10,7.5,21,8,10,5.5,12,18,13.0,20,39,20,0.650000,0.791775,-0.141775
49,Man United,EPL,11,9.0,24,5,10,6.5,11,19,15.5,21,43,16,0.738095,0.878385,-0.140290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,Stoke,EPL,10,5.0,13,19,11,2.5,27,10,7.5,21,23,46,0.357143,0.200000,0.157143
57,Northampton,FL1,12,5.0,9,19,13,3.5,26,11,8.5,25,20,45,0.340000,0.164948,0.175052
6,Birmingham,FLCH,13,6.0,8,13,12,1.5,22,5,7.5,25,13,35,0.300000,0.121234,0.178766
17,Burton,FLCH,13,3.0,11,27,12,5.0,16,6,8.0,25,17,43,0.320000,0.135173,0.184827


In [143]:
teams_q4 = ['Liverpool', 'Man United', 'Arsenal', 'Man City']
filtered_df = Eng17[Eng17['team'].isin(teams_q4)]
filtered_df.sort_values(by='difference')

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa,W,G,GF,GA,wpc,pyth,difference
49,Man United,EPL,11,9.0,24,5,10,6.5,11,19,15.5,21,43,16,0.738095,0.878385,-0.140290
46,Liverpool,EPL,11,8.5,21,4,10,6.5,20,27,15.0,21,48,24,0.714286,0.800000,-0.085714
2,Arsenal,EPL,10,8.5,25,10,11,5.0,16,13,13.5,21,38,26,0.642857,0.681132,-0.038275
48,Man City,EPL,10,9.5,36,7,11,10.5,5,25,20.0,21,61,12,0.952381,0.962743,-0.010362


In [176]:
#Which of the following teams had the highest value for away wins (awinvalue) for in the first half of the season?
teams_q6 = ['West Ham','Crystal Palace','Stoke','West Brom']
filtered_df6 = Eng17[Eng17['team'].isin(teams_q6)]
filtered_df6.sort_values(by='awinvalue')

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa,W,G,GF,GA,wpc,pyth,difference
30,Crystal Palace,EPL,11,5.0,14,18,10,2.5,14,4,7.5,21,18,32,0.357143,0.240356,0.116787
79,Stoke,EPL,10,5.0,13,19,11,2.5,27,10,7.5,21,23,46,0.357143,0.200000,0.157143
86,West Brom,EPL,11,4.5,10,15,10,2.5,13,5,7.0,21,15,28,0.333333,0.222993,0.110340
87,West Ham,EPL,9,4.0,10,14,11,3.0,24,12,7.0,20,22,38,0.350000,0.251037,0.098963


## Q5 Which of the following teams had the smallest difference between their win percentage and Pythagorean expectation in the second half of the season?

In [147]:
#First, creating a df with only the second half of the season (year > 2017)
half2 = EPL[EPL.Year > 2017]
half2

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Year
209,EPL,2018-01-01,Brighton,Bournemouth,2,2,D,2018
210,EPL,2018-01-01,Burnley,Liverpool,1,2,A,2018
211,EPL,2018-01-01,Everton,Man United,0,2,A,2018
212,EPL,2018-01-01,Leicester,Huddersfield,3,0,H,2018
213,EPL,2018-01-01,Stoke,Newcastle,0,1,A,2018
...,...,...,...,...,...,...,...,...
2031,FL2,2018-05-05,Lincoln,Yeovil,1,1,D,2018
2032,FL2,2018-05-05,Mansfield,Crawley Town,1,1,D,2018
2033,FL2,2018-05-05,Notts County,Luton,0,0,D,2018
2034,FL2,2018-05-05,Swindon,Accrington,3,0,H,2018


In [149]:
#We first identify whether the results was a win for the home team, the away team or a draw. We also create the counting varible.

half2['hwinvalue']=np.where(half2['FTR']=='H',1,np.where(half2['FTR']=='D',0.5,0))
half2['awinvalue'] = np.where(half2['FTR'] == 'A',1,np.where(half2['FTR']=='D',0.5,0))
half2['count']=1  
half2

/var/folders/0q/s7fj151s553_0czdxm4mtj680000gn/T/ipykernel_15273/4078126502.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  half2['hwinvalue']=np.where(half2['FTR']=='H',1,np.where(half2['FTR']=='D',0.5,0))
/var/folders/0q/s7fj151s553_0czdxm4mtj680000gn/T/ipykernel_15273/4078126502.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  half2['awinvalue'] = np.where(half2['FTR'] == 'A',1,np.where(half2['FTR']=='D',0.5,0))
/var/folders/0q/s7fj151s553_0czdxm4mtj680000gn/T/ipykernel_15273/4078126502.py:5: Sett

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Year,hwinvalue,awinvalue,count
209,EPL,2018-01-01,Brighton,Bournemouth,2,2,D,2018,0.5,0.5,1
210,EPL,2018-01-01,Burnley,Liverpool,1,2,A,2018,0.0,1.0,1
211,EPL,2018-01-01,Everton,Man United,0,2,A,2018,0.0,1.0,1
212,EPL,2018-01-01,Leicester,Huddersfield,3,0,H,2018,1.0,0.0,1
213,EPL,2018-01-01,Stoke,Newcastle,0,1,A,2018,0.0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...
2031,FL2,2018-05-05,Lincoln,Yeovil,1,1,D,2018,0.5,0.5,1
2032,FL2,2018-05-05,Mansfield,Crawley Town,1,1,D,2018,0.5,0.5,1
2033,FL2,2018-05-05,Notts County,Luton,0,0,D,2018,0.5,0.5,1
2034,FL2,2018-05-05,Swindon,Accrington,3,0,H,2018,1.0,0.0,1


In [151]:
#Once again, we have to create separate dfs to calculate home team and away team performance.

Enghome = half2.groupby(['HomeTeam','Div'])[['count','hwinvalue','FTHG','FTAG']].sum().reset_index()
Enghome = Enghome.rename(columns={'HomeTeam':'team','count':'Ph','FTHG':'FTHGh','FTAG':'FTAGh'})
Enghome

,team,Div,Ph,hwinvalue,FTHGh,FTAGh
0,AFC Wimbledon,FL1,11,6.5,14,13
1,Accrington,FL2,12,11.5,23,7
2,Arsenal,EPL,9,7.5,29,10
3,Aston Villa,FLCH,11,9.0,25,9
4,Barnet,FL2,11,6.5,14,11
...,...,...,...,...,...,...
87,West Ham,EPL,10,6.0,14,12
88,Wigan,FL1,10,7.0,16,7
89,Wolves,FLCH,11,8.5,22,8
90,Wycombe,FL2,10,6.0,16,17


In [153]:
#Now we create the away team results df

Engaway = half2.groupby('AwayTeam')[['count','awinvalue','FTHG','FTAG']].sum().reset_index()
Engaway = Engaway.rename(columns={'AwayTeam':'team','count':'Pa','FTHG':'FTHGa','FTAG':'FTAGa'})
Engaway

,team,Pa,awinvalue,FTHGa,FTAGa
0,AFC Wimbledon,11,4.5,14,13
1,Accrington,10,6.5,9,15
2,Arsenal,8,1.0,15,7
3,Aston Villa,10,5.5,9,14
4,Barnet,10,3.0,17,7
...,...,...,...,...,...
87,West Ham,8,3.0,18,12
88,Wigan,12,9.0,10,24
89,Wolves,10,6.0,11,13
90,Wycombe,11,9.5,9,19


In [155]:
#Merging the two dfs

Eng18 = pd.merge(Enghome, Engaway, on=['team'])
Eng18

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa
0,AFC Wimbledon,FL1,11,6.5,14,13,11,4.5,14,13
1,Accrington,FL2,12,11.5,23,7,10,6.5,9,15
2,Arsenal,EPL,9,7.5,29,10,8,1.0,15,7
3,Aston Villa,FLCH,11,9.0,25,9,10,5.5,9,14
4,Barnet,FL2,11,6.5,14,11,10,3.0,17,7
...,...,...,...,...,...,...,...,...,...,...
87,West Ham,EPL,10,6.0,14,12,8,3.0,18,12
88,Wigan,FL1,10,7.0,16,7,12,9.0,10,24
89,Wolves,FLCH,11,8.5,22,8,10,6.0,11,13
90,Wycombe,FL2,10,6.0,16,17,11,9.5,9,19


In [157]:
#Sum the results by home and away measures to get the team overall performance for the season

Eng18['W'] = Eng18['hwinvalue']+Eng18['awinvalue']
Eng18['G'] = Eng18['Ph']+Eng18['Pa']
Eng18['GF'] = Eng18['FTHGh']+Eng18['FTAGa']
Eng18['GA'] = Eng18['FTAGh']+Eng18['FTHGa']
Eng18

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa,W,G,GF,GA
0,AFC Wimbledon,FL1,11,6.5,14,13,11,4.5,14,13,11.0,22,27,27
1,Accrington,FL2,12,11.5,23,7,10,6.5,9,15,18.0,22,38,16
2,Arsenal,EPL,9,7.5,29,10,8,1.0,15,7,8.5,17,36,25
3,Aston Villa,FLCH,11,9.0,25,9,10,5.5,9,14,14.5,21,39,18
4,Barnet,FL2,11,6.5,14,11,10,3.0,17,7,9.5,21,21,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,West Ham,EPL,10,6.0,14,12,8,3.0,18,12,9.0,18,26,30
88,Wigan,FL1,10,7.0,16,7,12,9.0,10,24,16.0,22,40,17
89,Wolves,FLCH,11,8.5,22,8,10,6.0,11,13,14.5,21,35,19
90,Wycombe,FL2,10,6.0,16,17,11,9.5,9,19,15.5,21,35,26


In [159]:
#Creating wpc and Pythagorean Expectation

Eng18['wpc']=Eng18['W']/Eng18['G']
Eng18['pyth']=Eng18['GF']**2/(Eng18['GF']**2+Eng18['GA']**2)
Eng18

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa,W,G,GF,GA,wpc,pyth
0,AFC Wimbledon,FL1,11,6.5,14,13,11,4.5,14,13,11.0,22,27,27,0.500000,0.500000
1,Accrington,FL2,12,11.5,23,7,10,6.5,9,15,18.0,22,38,16,0.818182,0.849412
2,Arsenal,EPL,9,7.5,29,10,8,1.0,15,7,8.5,17,36,25,0.500000,0.674649
3,Aston Villa,FLCH,11,9.0,25,9,10,5.5,9,14,14.5,21,39,18,0.690476,0.824390
4,Barnet,FL2,11,6.5,14,11,10,3.0,17,7,9.5,21,21,28,0.452381,0.360000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,West Ham,EPL,10,6.0,14,12,8,3.0,18,12,9.0,18,26,30,0.500000,0.428934
88,Wigan,FL1,10,7.0,16,7,12,9.0,10,24,16.0,22,40,17,0.727273,0.847009
89,Wolves,FLCH,11,8.5,22,8,10,6.0,11,13,14.5,21,35,19,0.690476,0.772383
90,Wycombe,FL2,10,6.0,16,17,11,9.5,9,19,15.5,21,35,26,0.738095,0.644398


In [161]:
#New column for difference between wpc and pyth
Eng18['difference'] = Eng18['wpc']-Eng18['pyth']
Eng18.sort_values(by='difference')

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa,W,G,GF,GA,wpc,pyth,difference
2,Arsenal,EPL,9,7.5,29,10,8,1.0,15,7,8.5,17,36,25,0.500000,0.674649,-0.174649
46,Liverpool,EPL,8,7.0,24,6,9,5.0,8,12,12.0,17,36,14,0.705882,0.868633,-0.162750
8,Blackpool,FL1,10,6.5,17,8,11,5.0,11,11,11.5,21,28,19,0.547619,0.684716,-0.137097
3,Aston Villa,FLCH,11,9.0,25,9,10,5.5,9,14,14.5,21,39,18,0.690476,0.824390,-0.133914
88,Wigan,FL1,10,7.0,16,7,12,9.0,10,24,16.0,22,40,17,0.727273,0.847009,-0.119736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,Grimsby,FL2,10,4.0,8,11,11,2.5,23,8,6.5,21,16,34,0.309524,0.181303,0.128221
11,Bradford,FL1,10,4.5,10,13,11,2.5,21,7,7.0,21,17,34,0.333333,0.200000,0.133333
85,Watford,EPL,8,5.5,13,8,9,0.5,19,1,6.0,17,14,27,0.352941,0.211892,0.141049
40,Huddersfield,EPL,8,2.5,6,13,9,2.5,13,4,5.0,17,10,26,0.294118,0.128866,0.165252


In [163]:
teams_q5 = ['Bournemouth', 'Leicester', 'Stoke', 'Brighton']
filtered_df = Eng18[Eng18['team'].isin(teams_q5)]
filtered_df.sort_values(by='difference')

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa,W,G,GF,GA,wpc,pyth,difference
44,Leicester,EPL,9,5.0,12,8,8,2.5,20,13,7.5,17,25,28,0.441176,0.443577,-0.002401
79,Stoke,EPL,9,2.5,7,11,8,3.0,11,5,5.5,17,12,22,0.323529,0.229299,0.094230
10,Bournemouth,EPL,8,5.0,12,13,9,4.0,16,13,9.0,17,25,29,0.529412,0.426330,0.103082
13,Brighton,EPL,9,5.5,14,13,8,1.5,16,5,7.0,17,19,29,0.411765,0.300333,0.111432


In [182]:
#Q7 Which team had the largest gap between home points won (hwinvalue) and away points won (awinvalue) in the second half the season?

Eng18['pts diff'] = Eng18['hwinvalue']-Eng18['awinvalue']

teams_q7 = ['Brighton','Arsenal','Watford','West Ham']
filtered_df7 = Eng18[Eng18['team'].isin(teams_q7)]
filtered_df7.sort_values(by='pts diff')

,team,Div,Ph,hwinvalue,FTHGh,FTAGh,Pa,awinvalue,FTHGa,FTAGa,W,G,GF,GA,wpc,pyth,difference,pts diff
87,West Ham,EPL,10,6.0,14,12,8,3.0,18,12,9.0,18,26,30,0.500000,0.428934,0.071066,3.0
13,Brighton,EPL,9,5.5,14,13,8,1.5,16,5,7.0,17,19,29,0.411765,0.300333,0.111432,4.0
85,Watford,EPL,8,5.5,13,8,9,0.5,19,1,6.0,17,14,27,0.352941,0.211892,0.141049,5.0
2,Arsenal,EPL,9,7.5,29,10,8,1.0,15,7,8.5,17,36,25,0.500000,0.674649,-0.174649,6.5


In [186]:
#Q8 What was the correlation between win percentage and the Pythagorean expectation in the first half of the season?

Eng_full = pd.merge(Eng17, Eng18, on=['team'])
Eng_full

,team,Div_x,Ph_x,hwinvalue_x,FTHGh_x,FTAGh_x,Pa_x,awinvalue_x,FTHGa_x,FTAGa_x,...,FTHGa_y,FTAGa_y,W_y,G_y,GF_y,GA_y,wpc_y,pyth_y,difference_y,pts diff
0,AFC Wimbledon,FL1,12,4.5,11,17,12,4.5,14,9,...,14,13,11.0,22,27,27,0.500000,0.500000,0.000000,2.0
1,Accrington,FL2,11,7.0,19,12,13,7.0,18,19,...,9,15,18.0,22,38,16,0.818182,0.849412,-0.031230,5.0
2,Arsenal,EPL,10,8.5,25,10,11,5.0,16,13,...,15,7,8.5,17,36,25,0.500000,0.674649,-0.174649,6.5
3,Aston Villa,FLCH,12,8.5,17,10,13,6.5,14,16,...,9,14,14.5,21,39,18,0.690476,0.824390,-0.133914,3.5
4,Barnet,FL2,12,4.5,10,14,13,3.0,23,15,...,17,7,9.5,21,21,28,0.452381,0.360000,0.092381,3.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,West Ham,EPL,9,4.0,10,14,11,3.0,24,12,...,18,12,9.0,18,26,30,0.500000,0.428934,0.071066,3.0
88,Wigan,FL1,13,10.0,21,4,11,8.5,8,28,...,10,24,16.0,22,40,17,0.727273,0.847009,-0.119736,-2.0
89,Wolves,FLCH,12,10.0,25,10,13,10.0,10,22,...,11,13,14.5,21,35,19,0.690476,0.772383,-0.081907,2.5
90,Wycombe,FL2,13,8.5,27,18,12,6.0,16,17,...,9,19,15.5,21,35,26,0.738095,0.644398,0.093698,-3.5


In [188]:
keyvars = Eng_full[['wpc_y','wpc_x','pyth_y','pyth_x']]
keyvars.corr()

,wpc_y,wpc_x,pyth_y,pyth_x
wpc_y,1.000000,0.601684,0.942746,0.615636
wpc_x,0.601684,1.000000,0.618430,0.959210
pyth_y,0.942746,0.618430,1.000000,0.651551
pyth_x,0.615636,0.959210,0.651551,1.000000
